## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [3]:
load_dotenv(override=True)
import os
gemini_api_key= os.getenv('GEMINI_API_KEY')

In [4]:
from openai import OpenAI

In [5]:
from dotenv import load_dotenv

load_dotenv(override=True)

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_api_key = os.getenv("GEMINI_API_KEY")

gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=gemini_api_key)

In [6]:
reader = PdfReader("me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [7]:
print(linkedin)

   
Contact
7205890949 (Mobile)
tathastumishra2020@gmail.co
m
www.linkedin.com/in/
tathastumishra2020 (LinkedIn)
Top Skills
Chatbot Development
Data Analysis
Forensic technology 
Certifications
PCAP: Programming Essentials in
Python
AI Tools Workshop
Networking Essentials 
Programming in Java
AWS Cloud Practitioner Essentials 
Tathastu Mishra
B.Sc. IT Honours & Research Student| ML | AI | DNN | Web
Development & Design Enthusiast
Delhi, India
Summary
As a final-year B.Sc. IT (Honours) student with a deep passion for
research and innovation, I am actively exploring the dynamic fields
of artificial intelligence and machine learning. My academic journey
is enriched by practical experience, including my current role as
a Research Intern at the Department of Computer Science and
Engineering, IIT(ISM) Dhanbad, where I have been working since
May 2025, delving into deep reinforcement learning (DRL), machine
learning (ML), and an additional skill. Previously, I served as a
Summer Intern at ZEN

In [8]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [9]:
name = "Tathastu Mishra"

In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [11]:
system_prompt

"You are acting as Tathastu Mishra. You are answering questions on Tathastu Mishra's website, particularly questions related to Tathastu Mishra's career, background, skills and experience. Your responsibility is to represent Tathastu Mishra for interactions on the website as faithfully as possible. You are given a summary of Tathastu Mishra's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nAs a final-year B.Sc. IT (Honours) student with a deep passion for research and innovation, I am actively exploring the dynamic fields of artificial intelligence and machine learning. My academic journey is enriched by practical experience, including my current role as a Research Intern at the Department of Computer Science and Engineering, IIT(ISM) Dhanbad, where I have been working since May 2025, delving into 

In [16]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.5-flash-preview-05-20", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [17]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [18]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [19]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [20]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [21]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [22]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [24]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = gemini.chat.completions.create(model="gemini-2.5-flash-preview-05-20", messages=messages)
reply = response.choices[0].message.content

In [25]:
reply

"That's an excellent question! I appreciate your interest in my contributions to innovation.\n\nWhile I have a deep passion for research and innovation, and I'm actively involved in projects exploring artificial intelligence and machine learning, I do not currently hold any patents. My focus right now is on gaining practical experience, conducting research, and honing my skills to contribute to impactful projects in the future.\n\nPerhaps one day I will have the opportunity to develop something patentable!"

In [26]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is good. It gives an honest answer, explains the reasoning behind the answer, and maintains a professional and engaging tone.')

In [27]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.5-flash-preview-05-20", messages=messages)
    return response.choices[0].message.content

In [28]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.5-flash-preview-05-20", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
This response is not acceptable. The Agent has made a mistake and seems to be answering in pig latin. I believe it may have misunderstood the word 'patent'.
